# ETL para Redis

## Instalações

In [1]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


## Importações

In [2]:
import os
import sys
from tqdm import tqdm

from pyspark.sql import SparkSession

from neo4j_consulta import generate_recommendations

## Constantes

In [3]:
DATASET_PATH = '../../dataset'

USER_PATH = os.path.join(DATASET_PATH, 'users-details-2023.csv')
USER_FILTERED_PATH = os.path.join(DATASET_PATH, 'user-filtered.csv')
ANIME_FILTERED_PATH = os.path.join(DATASET_PATH, 'anime-filtered.csv')

## Código para resolver um problema de versão encontrada pelo PySpark

A resolução do erro foi encontrada em uma resposta no [StackOverflow](https://stackoverflow.com/a/65010346)

In [4]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Configurando inicialização do "conector" Neo4j spark

In [5]:
spark = SparkSession.\
        builder.\
        config("spark.jars", "../../spark-redis/target/spark-redis_2.12-3.1.0-SNAPSHOT-jar-with-dependencies.jar")\
        .master('local[*]') \
        .config('spark.sql.execution.arrow.pyspark.enabled', True) \
        .config('spark.sql.session.timeZone', 'UTC') \
        .config('spark.driver.memory','32G') \
        .config('spark.ui.showConsoleProgress', True) \
        .config('spark.sql.repl.eagerEval.enabled', True) \
        .getOrCreate()

## Lendo o arquivo users-details-2023.csv

In [6]:
df_user = spark.read.csv(USER_PATH, inferSchema=True, header=True, escape='"', multiLine=True)
df_user = df_user.select(['mal ID', 'username', 'gender', 'days watched', 'completed', 'dropped', 'location'])

## Lendo o arquivo anime-filtered.csv

In [7]:
df_anime_filtered = spark.read.csv(ANIME_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)
df_anime_filtered = df_anime_filtered.select(['anime_id', 'name'])

## Lendo o arquivo user_filtered.csv

In [9]:
df_user_filtered = spark.read.csv(USER_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)
df_user_filtered = df_user_filtered.select(['anime_id', 'user_id'])

## Gerando os agregados

### Coletando as recomendações para os usuários

In [10]:
recommendations = generate_recommendations()

In [11]:
print(recommendations[148])

[{'id': 1735, 'name': 'Naruto: Shippuuden'}, {'id': 1535, 'name': 'Death Note'}, {'id': 5114, 'name': 'Fullmetal Alchemist: Brotherhood'}]


### Criando os agregados iniciais

In [12]:
agregados = {}

users = df_user.collect()
qnt_users = len(users)

for user in tqdm(users, total=qnt_users):
    user_id = user['mal ID']
    
    agregados[user_id] = {}
    agregados[user_id]['username'] = user['username']
    agregados[user_id]['gender'] = user['gender']
    agregados[user_id]['days watched'] = user['days watched']
    agregados[user_id]['completed'] = user['completed']
    agregados[user_id]['dropped'] = user['dropped']
    agregados[user_id]['location'] = user['location']
    agregados[user_id]['watched animes'] = []
    
    if user_id in recommendations:
        agregados[user_id]['recomendations'] = recommendations[user_id]

100%|██████████| 731290/731290 [00:07<00:00, 102841.81it/s]


In [13]:
agregados[1]

{'username': 'Xinil',
 'gender': 'Male',
 'days watched': 142.3,
 'completed': 233.0,
 'dropped': 93.0,
 'location': 'California',
 'watched animes': [],
 'recomendations': [{'id': 5114, 'name': 'Fullmetal Alchemist: Brotherhood'},
  {'id': 9253, 'name': 'Steins;Gate'},
  {'id': 2904, 'name': 'Code Geass: Hangyaku no Lelouch R2'}]}

### Adicionando aos agregados os animes que os usuários assistiram

In [15]:
consumed_animes = df_user_filtered.join(df_anime_filtered, 'anime_id').collect()
qnt_consumed_animes = len(consumed_animes)

for consumed_anime in tqdm(consumed_animes, total=consumed_animes):
    agregados[consumed_anime['user_id']].append({
        'id': consumed_anime['anime_id'],
        'name': consumed_anime['title']
    })

ConnectionRefusedError: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente

In [ ]:
print(agregados[1])

### Tranformando agregados de dicionário para DataFrame Spark

### Salvando os dados no Redis